In [18]:
run ~/Projects/replab/replab_addpaths % replace by your own path

# Finding an upper bound on CHSH, symmetrized version
During the lecture, we found a form of the SDP that had only one optimization variable. We'll now see how to symmetrize the problem completely.

We first declare the one and only variable, before constructing the moment matrix.

In [19]:
y = sdpvar; % y is actually y_A0B0, but we save space by identifying y with it
X = [ 1  0  0  0  0
      0  1  0  y  y
      0  0  1  y -y
      0  y  y  1  0
      0  y -y  0  1];
I_CHSH = 4*y; % it is <A0B0> + <A0B1> + <A1B0> - <A1B1>, and y_A1B1 = -y_A0B0.
optimize(X >= 0, -I_CHSH, sdpsettings('verbose', 0)); % sign change to maximize, and we don't show solver out
double(I_CHSH)


ans =

    2.8284



To help with symmetrization, we'll use an equivalent form of the constraint X, and check the result.

In [20]:
C = eye(5); % identity
A = [0  0  0  0  0
     0  0  0  1  1
     0  0  0  1 -1
     0  1  1  0  0
     0  1 -1  0  0];
X = C + A*y; % equivalent to the constraint above
optimize(X >= 0, -I_CHSH); % sign change to maximize
double(I_CHSH)


 num. of constraints =  1
 dim. of sdp    var  =  5,   num. of sdp  blk  =  1
*******************************************************************
   SDPT3: Infeasible path-following algorithms
*******************************************************************
 version  predcorr  gam  expon  scale_data
   HKM      1      0.000   1        0    
it pstep dstep pinfeas dinfeas  gap      prim-obj      dual-obj    cputime
-------------------------------------------------------------------
 0|0.000|0.000|8.0e-01|6.2e+00|5.0e+02| 5.000000e+01  0.000000e+00| 0:0:00| chol  1  1 
 1|1.000|1.000|0.0e+00|6.9e-02|4.3e+01| 3.984640e+01  4.400000e-01| 0:0:00| chol  1  1 
 2|0.933|1.000|1.8e-16|6.9e-03|2.9e+00| 3.700415e+00  8.665596e-01| 0:0:00| chol  1  1 
 3|1.000|0.949|1.8e-16|1.0e-03|2.8e-01| 3.024905e+00  2.752468e+00| 0:0:00| chol  1  1 
 4|0.987|0.988|1.8e-16|8.0e-05|3.4e-03| 2.830875e+00  2.827762e+00| 0:0:00| chol  1  1 
 5|0.989|0.989|0.0e+00|7.7e-06|3.8e-05| 2.828454e+00  2.828448e+00| 0:

Now, we want to find a change of basis that brings X into a block-diagonal form. This is easier to check on the basis matrices `C` and `A`, i.e. we want to find `U` such that `U*C*U'` and `U*A*U'` are block-diagonal. Let us use RepLAB for that.
First, remark that the moment matrix is invariant under the following signed permutations:

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, B_0, B_1, A_0, A_1 \right) $

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, -A_0, -A_1, -B_0, -B_1 \right) $

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, A_1, A_0, B_0, -B_1 \right) $

We now show how to use RepLAB to find the change of basis for the symmetry group that includes only the first symmetry.

In [21]:
g1 = [1 4 5 2 3]; % permutation of parties
g2 = [1 -2 -3 -4 -5]; % sign flip everywhere, note the signed permutation convention
g3 = [1 3 2 4 -5]; % additional symmetry

We build the symmetry group from those generators, which are signed permutations on five elements. For the permutation of parties only, the order (=size) of the group is 2. When adding the other symmetries, the order of the group should be 16.

In [22]:
nElements = 5;
G = replab.signed.Permutations(nElements).subgroup({g1 g2 g3});
G.order

ans =
    16


If necessary, we can also expand the group elements (use `G.elements.at(2)` to get the second element for example).

In [23]:
G.elements


ans = 

Indexed family of 16 elements
 1 = [1, 2, 3, 4, 5]    
 2 = [1, -3, -2, -4, 5] 
 3 = [1, 2, -3, 5, 4]   
 4 = [1, 3, -2, -5, 4]  
 5 = [1, 4, 5, 2, 3]    
 6 = [1, -5, -4, -2, 3] 
 7 = [1, 4, -5, 3, 2]   
 8 = [1, 5, -4, -3, 2]  
 9 = [1, -4, 5, -3, -2] 
10 = [1, -5, 4, 3, -2]  
11 = [1, -4, -5, -2, -3]
12 = [1, 5, 4, 2, -3]   
13 = [1, -2, 3, -5, -4] 
14 = [1, -3, 2, 5, -4]  
15 = [1, -2, -3, -4, -5]
16 = [1, 3, 2, 4, -5]   


Now, the representatio we need is composed of signed permutation matrices; as we are considering a group of signed permutations, this is the group *natural representation*. Representations in RepLAB are described using the images of the generators. By calling `G.sampleUniformly`, we get random elements from the group, and by calling `rep.image(g)` we get the image of a group element.

In [24]:
rep = G.definingRep


rep = 

Orthogonal real representation of dimension 5
    dimension: 5                                                                              
        field: 'R'                                                                            
        group: replab.signed.PermutationSubgroup                                              
inverseImages: 1 x 3 cell                                                                     
    isUnitary: true                                                                           
    images{1}: [1, 0, 0, 0, 0; 0, 0, 0, 1, 0; 0, 0, 0, 0, 1; 0, 1, 0, 0, 0; 0, 0, 1, 0, 0]    
    images{2}: [1, 0, 0, 0, 0; 0, -1, 0, 0, 0; 0, 0, -1, 0, 0; 0, 0, 0, -1, 0; 0, 0, 0, 0, -1]
    images{3}: [1, 0, 0, 0, 0; 0, 0, 1, 0, 0; 0, 1, 0, 0, 0; 0, 0, 0, 1, 0; 0, 0, 0, 0, -1]   


In [25]:
g = G.sampleUniformly
full(rep.image(g)) % we use full, as signed permutation representations return sparse matrices


g =

     1    -3     2     5    -4


ans =

     1     0     0     0     0
     0     0     1     0     0
     0    -1     0     0     0
     0     0     0     0    -1
     0     0     0     1     0



Now, we decompose the representation into irreducible components. For that, we call `rep.decomposition`. In the answer I(m)xR(d), we express that the component has $m$ copies (multiplicity) of a `R`eal representation of dimension `d`. We then play with the indices of the component, and copy.

Hint: explore `rep.decomposition.component(i).irrep(j)` for $(i,j) = (1,1), (2,1), (3,1)$.

In [26]:
rep.decomposition


ans = 

Subrepresentation
   dimension: 5                                            
       field: 'R'                                          
       group: replab.signed.PermutationSubgroup            
   irrepInfo: []                                           
   isUnitary: true                                         
   niceBasis: []                                           
      parent: Orthogonal real representation of dimension 5
           U: 5 x 5 double                                 
component(1): Isotypic component R(1)                      
component(2): Isotypic component R(2)                      
component(3): Isotypic component R(2)                      


In [27]:
rep.decomposition.component(1).irrep(1)


ans = 

Real irreducible subrepresentation of real type
dimension: 1                                            
    field: 'R'                                          
    group: replab.signed.PermutationSubgroup            
irrepInfo: irreducible, '1', real of real type          
isUnitary: true                                         
niceBasis: replab.NiceBasis                             
   parent: Orthogonal real representation of dimension 5
   U(1,:): [1, 0, 0, 0, 0]                              


We now ask for the change of basis matrix, and verify that `A` and `C` block-diagonalize.

In [28]:
U = full(rep.decomposition.U)
U*C*U'
U*A*U'


U =

    1.0000         0         0         0         0
   -0.0000    0.2708    0.6532    0.6534   -0.2704
   -0.0000    0.6532   -0.2708    0.2704    0.6534
    0.0000   -0.7071   -0.0000    0.5000    0.5000
   -0.0000    0.0000    0.7071   -0.5000    0.5000


ans =

    1.0000   -0.0000   -0.0000    0.0000   -0.0000
   -0.0000    1.0000    0.0000   -0.0000   -0.0000
   -0.0000    0.0000    1.0000    0.0000   -0.0000
    0.0000   -0.0000    0.0000    1.0000   -0.0000
   -0.0000   -0.0000   -0.0000   -0.0000    1.0000


ans =

         0         0         0         0         0
         0    1.4142    0.0000   -0.0000    0.0000
         0    0.0000    1.4142    0.0000    0.0000
         0   -0.0000    0.0000   -1.4142    0.0000
         0    0.0000    0.0000    0.0000   -1.4142



Once we have found the change of basis matrix, we transform our moment matrix `X`, and naturally it should have a block diagonal structure following the one of `C` and `A` -- for that, it may be necessary to kill small off-block coefficients.

In [29]:
X = (U*C*U') + (U*A*U')*y; % equivalent to the constraint above
optimize(X >= 0, -I_CHSH); % sign change to maximize
double(I_CHSH)


 num. of constraints =  1
 dim. of sdp    var  =  5,   num. of sdp  blk  =  1
*******************************************************************
   SDPT3: Infeasible path-following algorithms
*******************************************************************
 version  predcorr  gam  expon  scale_data
   HKM      1      0.000   1        0    
it pstep dstep pinfeas dinfeas  gap      prim-obj      dual-obj    cputime
-------------------------------------------------------------------
 0|0.000|0.000|8.0e-01|6.2e+00|5.0e+02| 5.000000e+01  0.000000e+00| 0:0:00| chol  1  1 
 1|1.000|1.000|1.8e-15|6.9e-02|4.3e+01| 3.984640e+01  4.400000e-01| 0:0:00| chol  1  1 
 2|0.933|1.000|7.1e-16|6.9e-03|2.9e+00| 3.700415e+00  8.665596e-01| 0:0:00| chol  1  1 
 3|1.000|0.949|8.9e-17|1.0e-03|2.8e-01| 3.024905e+00  2.752468e+00| 0:0:00| chol  1  1 
 4|0.987|0.988|1.8e-16|8.0e-05|3.4e-03| 2.830875e+00  2.827762e+00| 0:0:00| chol  1  1 
 5|0.989|0.989|0.0e+00|7.7e-06|3.8e-05| 2.828454e+00  2.828448e+00| 0:

As an exercice, recover an algebraic basis from `U`, guess the form of the semidefinite program when fully block-diagonal (i.e. for the full group of CHSH symmetries).
**Remark that now the problem can be solved by hand to recover the $2\sqrt{2}$ bound, as the SDP matrix is fully diagonal, and thus corresponds to linear inequalities.**